<a href="https://colab.research.google.com/github/merdandt/MagicBook/blob/main/MagicBook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Package installatoin

### Check if you have an NVIDIA GPU


In [ ]:
# Note: If this returns "command not found", then GPU-based algorithms via cuGraph are unavailable

!nvidia-smi
!nvcc --version

/bin/bash: line 1: nvidia-smi: command not found
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [2]:
!pip install nx_arangodb arango-datasets langchain_community langchain-openai langchain-google-genai gradio PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.1/415.1 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 8

### Imports

In [39]:
from langchain_community.graphs.arangodb_graph import ArangoGraph
from langchain_community.chains.graph_qa.arangodb import ArangoGraphQAChain
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain.agents import AgentExecutor, Tool
from langchain.tools import BaseTool, tool
from langchain.chains import LLMChain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage


import networkx as nx

import re
from typing import List, Dict, Any
import nx_arangodb as nxadb
from arango import ArangoClient
import os, logging, PyPDF2


import pandas as pd
import gradio as gr

# Configure logging to show debug output in the console
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

### Set environment variables for ArangoDB connection

In [20]:
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')

os.environ['DATABASE_HOST'] = 'https://0e04-75-148-99-49.ngrok-free.app'
os.environ['DATABASE_USERNAME'] = 'root'
os.environ['DATABASE_PASSWORD'] = 'openSesame'       # Replace with your actual database name
os.environ['DATABASE_NAME'] = '_system'


### Initialize the db

In [21]:
client = ArangoClient(hosts="https://0e04-75-148-99-49.ngrok-free.app")
db = client.db("_system", username="root", password="openSesame", verify=False)

print(db)

<StandardDatabase _system>


### Initialize LLM

In [42]:
book_llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    api_key=os.environ.get("GEMINI_API_KEY"),
    temperature=0
  )

chat_llm = ChatOpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    model="gpt-4o",
    temperature=0
  )

In [38]:
def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file with no length constraints."""
    logging.debug("Starting PDF text extraction from file: %s", pdf_path)
    text = ""
    try:
        pdf_reader = PyPDF2.PdfReader(pdf_path)
        logging.debug("Opened PDF file successfully. Total pages: %d", len(pdf_reader.pages))
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            page_text = page.extract_text()
            # Log the first 100 characters from the page's text for debugging
            logging.debug("Page %d extracted text (first 100 chars): %s", page_num + 1, (page_text or "")[:100])
            if page_text:
                text += page_text + "\n"
            else:
                logging.debug("No text found on page %d", page_num + 1)
    except Exception as e:
        error_msg = f"Error extracting text from PDF: {str(e)}"
        logging.error(error_msg)
        return error_msg

    logging.debug("Completed PDF text extraction. Total extracted text length: %d characters", len(text))
    return text

def ask_book(question_input):
    """Ask a question about the book."""

def analyze_book_with_gemini(pdf_file):
    """Extract text from PDF and send it to Gemini for analysis."""
    logging.debug("Starting analysis of uploaded PDF file.")
    if pdf_file is None:
        logging.warning("No file uploaded.")
        return "No file uploaded. Please upload a PDF file."

    try:
        # Extract text from the PDF file
        logging.debug("Extracting text from PDF file: %s", pdf_file.name)
        text = extract_text_from_pdf(pdf_file.name)

        if text.startswith("Error"):
            logging.error("Error during PDF text extraction: %s", text)
            return text

        # Initialize Gemini model
        logging.debug("Initializing Gemini model: gemini-2.0-flash")


        # Create prompt with the extracted text
        prompt = f"""
        The following is text extracted from a book. Please analyze this and tell me:
        1. What is the title of this book?
        2. Who is the author?
        3. What is the main subject or genre?
        4. Provide a brief summary of what the book is about.

        Here's the text:
        {text}
        """
        # Log a snippet of the prompt for debugging
        logging.debug("Constructed prompt (first 200 characters): %s", prompt[:200])

        # Get response from Gemini by invoking with the prompt string
        logging.debug("Sending prompt to Gemini model.")
        response = book_llm.invoke(prompt)
        logging.debug("Received response from Gemini model: %s", response.content)

        return response.content

    except Exception as e:
        error_msg = f"Error analyzing book with Gemini: {str(e)}"
        logging.error(error_msg)
        return error_msg


/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:285: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://31ae420657acb54919.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
